In [13]:
%pip install transformers torch accelerate datasets evaluate numpy pandas scikit-learn

8126.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [17]:
import pandas as pd
import cleaning_tool as ct

# Load your dataset
df = pd.read_csv('cleaned_output.csv')
df = df.dropna()
df = ct.filter_single_users(dataframe=df, min_pull=5)
print(df.head())
df


   number                                              title  assignee  \
0       3                       omnisharp includ linux build   jrieken   
1       4                  c# bracket insertion, indent work  alexdima   
2       5                          welcome.md packag wrongli   isidorn   
3       6  window - omnisharp provid intellisens strong n...   jrieken   
4       8                      flash open workspaces, reload   jrieken   

                                                body  
0  our linux build machin includ csharp-o/**bin**...  
1      type (\n\nexpect => close )\nactual => noth\n  
2  vscode > help > show welcom > work\n\nif i loo...  
3  upgrad rc1 dnx/runtim\n\n```bash\ngit clone ht...  
4  start code\nset dark theme*other than the defa...  


,number,title,assignee,body
0,3,omnisharp includ linux build,jrieken,our linux build machin includ csharp-o/**bin**...
1,4,"c# bracket insertion, indent work",alexdima,type (\n\nexpect => close )\nactual => noth\n
2,5,welcome.md packag wrongli,isidorn,vscode > help > show welcom > work\n\nif i loo...
3,6,window - omnisharp provid intellisens strong n...,jrieken,upgrad rc1 dnx/runtim\n\n```bash\ngit clone ht...
4,8,"flash open workspaces, reload",jrieken,start code\nset dark theme*other than the defa...
...,...,...,...,...
132,196,move gulpfile.* /build/ folder,joaomoreno,would nice we'd less clutter root folder singl...
133,197,[debug] consol http link match incorrectli,isidorn,vs code show follow output console:\n\n\n\ngoo...
134,198,keybind use bracket (eg. indent) work danish k...,alexdima,"the reason like enter key layout, use 'option'..."
135,199,instal extension: order extens list,joaomoreno,when tri instal yeoman gener with:\n`ext insta...


In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode the assignee names
label_encoder = LabelEncoder()
df['assignee_encoded'] = label_encoder.fit_transform(df['assignee'])

df['input_text'] = "<#TITLE-START#> " + df['title'] + " <#TITLE-END#> <#BODY-START#> " + df['body'] + " <#BODY-END#>"


# Split into input features (titles) and labels (encoded assignees)
titles = df['input_text'].tolist()
labels = df['assignee_encoded'].tolist()

In [23]:
titles

['<#TITLE-START#> omnisharp includ linux build <#TITLE-END#> <#BODY-START#> our linux build machin includ csharp-o/**bin**folder.\n1. run`scripts/npm.sh install`-> csharp-o/**bin**folder nice get creat linux machine.\n2. run`gulp vscode-linux-x64`also nice creat csharp-o/**bin**folder linux machine.\n\nsometh strang build machin\n <#BODY-END#>',
 '<#TITLE-START#> c# bracket insertion, indent work <#TITLE-END#> <#BODY-START#> type (\n\nexpect => close )\nactual => noth\n <#BODY-END#>',
 '<#TITLE-START#> welcome.md packag wrongli <#TITLE-END#> <#BODY-START#> vscode > help > show welcom > work\n\nif i look content vscode packag i see top level resourc folder contain welcome.md. thi look badli place\n <#BODY-END#>',
 '<#TITLE-START#> window - omnisharp provid intellisens strong name assembl <#TITLE-END#> <#BODY-START#> upgrad rc1 dnx/runtim\n\n```bash\ngit clone https://github.com/natemcmaster/test-vscode-strong-name\ncd test-vscode-strong-name\ndnu restore\ncode .\n```\n\npick test projec

In [37]:
from transformers import AutoTokenizer

# Choose a model, e.g., 'distilbert-base-uncased'
# model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input titles
inputs = tokenizer(titles, padding=True, truncation=True, return_tensors='pt', max_length=128)
# inputs = tokenizer(titles, padding=True, truncation=True, return_tensors='pt', max_length=128).to("cuda")


/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
from datasets import Dataset

# Create a Hugging Face dataset
dataset = Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'labels': labels
})

# Split the dataset into training and validation sets
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']

len(labels)


115

In [39]:
import evaluate
import numpy as np

# Load metric functions
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1['f1'],
    }
    

7

In [40]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 33%|███▎      | 6/18 [00:01<00:02,  5.47it/s]/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

{'eval_loss': 1.956424593925476, 'eval_accuracy': 0.17391304347826086, 'eval_precision': 0.05124223602484472, 'eval_recall': 0.17391304347826086, 'eval_f1': 0.0782608695652174, 'eval_runtime': 0.0901, 'eval_samples_per_second': 255.275, 'eval_steps_per_second': 11.099, 'epoch': 1.0}


 67%|██████▋   | 12/18 [00:02<00:01,  5.29it/s]/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

 67%|██████▋   | 12/18 [00:02<00:01,  5.29it/s]

{'eval_loss': 1.9487797021865845, 'eval_accuracy': 0.21739130434782608, 'eval_precision': 0.08009153318077802, 'eval_recall': 0.21739130434782608, 'eval_f1': 0.10396975425330812, 'eval_runtime': 0.1156, 'eval_samples_per_second': 198.914, 'eval_steps_per_second': 8.648, 'epoch': 2.0}


100%|██████████| 18/18 [00:03<00:00,  5.20it/s]/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

100%|██████████| 18/18 [00:04<00:00,  3.82it/s]

{'eval_loss': 1.9498947858810425, 'eval_accuracy': 0.2608695652173913, 'eval_precision': 0.12087912087912088, 'eval_recall': 0.2608695652173913, 'eval_f1': 0.15703324808184146, 'eval_runtime': 0.1029, 'eval_samples_per_second': 223.577, 'eval_steps_per_second': 9.721, 'epoch': 3.0}
{'train_runtime': 4.7125, 'train_samples_per_second': 58.568, 'train_steps_per_second': 3.82, 'train_loss': 1.9143558078342013, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=1.9143558078342013, metrics={'train_runtime': 4.7125, 'train_samples_per_second': 58.568, 'train_steps_per_second': 3.82, 'total_flos': 9141065524224.0, 'train_loss': 1.9143558078342013, 'epoch': 3.0})

In [41]:
trainer.evaluate()


  0%|          | 0/1 [00:00<?, ?it/s]/Users/casarf/Documents/USI/Master/semester3/SoftwareAnalytics/project/bug-triaging/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 1/1 [00:00<00:00, 80.72it/s]


{'eval_loss': 1.9498947858810425,
 'eval_accuracy': 0.2608695652173913,
 'eval_precision': 0.12087912087912088,
 'eval_recall': 0.2608695652173913,
 'eval_f1': 0.15703324808184146,
 'eval_runtime': 0.16,
 'eval_samples_per_second': 143.788,
 'eval_steps_per_second': 6.252,
 'epoch': 3.0}

## CUDA

In [20]:
def predict_assignee(title):
    inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=128).to('cuda')
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax(dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# Example prediction
print(predict_assignee("Side bar always showing when opening VS Code"))


sbatten


## MACOS

In [52]:
import torch
from scipy.special import softmax

# Check if MPS is available
device = torch.device('mps') if torch.has_mps else torch.device('cpu')

# Move the model to the correct device
model.to(device)

# def predict_assignee(title):
#     # Tokenize the input
#     inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=128)

#     # Move input tensors to the correct device
#     inputs = {key: value.to(device) for key, value in inputs.items()}

#     # Get model output
#     outputs = model(**inputs)

#     # Get the predicted class
#     predicted_class = outputs.logits.argmax(dim=1).item()

#     # Return the assignee name
#     return label_encoder.inverse_transform([predicted_class])[0]

# Example prediction
# print(predict_assignee("Change the name"))

def predict_assignee(title, body):
    # Concatenate title and body
    combined_input = title + " " + body
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Move input tensors to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Get model output
    outputs = model(**inputs)
    
    # Get logits
    logits = outputs.logits.detach().cpu().numpy()[0]
    
    # Get probabilities using softmax
    probabilities = softmax(logits)
    
    # Create a list of (assignee, probability) pairs
    assignee_probs = list(zip(label_encoder.classes_, probabilities))
    
    # Sort by probability in descending order
    ranked_assignees = sorted(assignee_probs, key=lambda x: x[1], reverse=True)
    
    return ranked_assignees

# Example prediction with both title and body
ranked_list = predict_assignee("Side bar always showing when opening VS Code", "When opening VS Code, the sidebar is always visible")

for index, (assignee, probability) in enumerate(ranked_list):
    print(f"{index + 1}: {assignee}: {probability:.2f}")



1: aeschli: 0.16
2: jrieken: 0.16
3: bpasero: 0.16
4: joaomoreno: 0.14
5: alexdima: 0.14
6: weinand: 0.13
7: isidorn: 0.11


/var/folders/2w/_w5hf_g91nb_rfqwyq6pv2mr0000gn/T/ipykernel_10551/1114867945.py:5: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device('mps') if torch.has_mps else torch.device('cpu')
